In [8]:
import time
import os
import csv
from dotenv import load_dotenv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
PAGE_URL = 'https://m.facebook.com/groups/322977734828852'
OUTPUT_CSV_FILE = 'Facebook_group_post_urls_last5y.csv'
PROFILE_PATH = r'C:\chrome-profiles\fb-pipeline-stage1-persistent'

def login_to_facebook(driver):
    driver.get("https://m.facebook.com")
    time.sleep(3)
    selectors = [
        "button[data-cookiebanner='accept_button_dialog']",
        "button[title='Allow all cookies']",
        "button[title='Accept All']",
        "button[aria-label='Allow all cookies']"
    ]
    for s in selectors:
        btns = driver.find_elements(By.CSS_SELECTOR, s)
        if btns and btns[0].is_displayed():
            btns[0].click()
            time.sleep(2)
            break
    email = driver.find_elements(By.ID, "m_login_email")
    pwd = driver.find_elements(By.ID, "m_login_password")
    if not email:
        email = driver.find_elements(By.ID, "email")
        pwd = driver.find_elements(By.ID, "pass")
    if email and pwd:
        email[0].clear()
        email[0].send_keys(FACEBOOK_EMAIL)
        pwd[0].send_keys(FACEBOOK_PASSWORD)
        pwd[0].send_keys(Keys.RETURN)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[aria-label='Menu'], a[aria-label='Home'], a[aria-label='Search']")))

def collect_group_posts_last_5y(driver, group_url):
    cutoff_epoch = int(time.time() - 5 * 365 * 24 * 60 * 60)
    driver.get(group_url)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='main'], #m_group_stories_container, article")))
    time.sleep(3)
    group_id = group_url.rstrip('/').split('/')[-1]
    urls = set()
    prev_len = 0
    stagnant = 0
    reached_old = False
    loops = 0
    while True:
        data = driver.execute_script("""
            var gid = arguments[0];
            var anchors = Array.from(document.querySelectorAll("a[href*='/groups/']"));
            var out = [];
            for (var i=0;i<anchors.length;i++){
                var a = anchors[i];
                var href = a.getAttribute("href") || "";
                if(!href) continue;
                if(href.indexOf("/groups/"+gid+"/") === -1 && href.indexOf("story.php") === -1) continue;
                if(href.indexOf("/reel") !== -1 || href.indexOf("/reels/") !== -1 || href.indexOf("/videos/") !== -1 || href.indexOf("/watch/") !== -1 || href.indexOf("video_id=") !== -1) continue;
                if(href.indexOf("/permalink/") === -1 && href.indexOf("/posts/") === -1 && href.indexOf("story.php") === -1) continue;
                if(href.indexOf("http")!==0){ href = location.origin + href; }
                href = href.split("?")[0];
                var el = a.closest("article") || a.closest("div");
                var ut = null;
                if(el){
                    var t = el.querySelector("abbr[data-utime], time[data-utime]");
                    if(t){ ut = parseInt(t.getAttribute("data-utime")); }
                    if(!ut){
                        var t2 = el.querySelector("time[datetime], abbr[datetime]");
                        if(t2 && t2.getAttribute("datetime")){
                            var ms = Date.parse(t2.getAttribute("datetime"));
                            if(!isNaN(ms)) ut = Math.floor(ms/1000);
                        }
                    }
                }
                out.push([href, ut]);
            }
            return out;
        """, group_id)
        for href, ut in data:
            if ut is None or ut >= cutoff_epoch:
                urls.add(href)
        if any((ut is not None and ut < cutoff_epoch) for _, ut in data):
            reached_old = True
        see_more = driver.find_elements(By.XPATH, "//div[@role='button' and (contains(., 'See more') or contains(., 'See More'))]")
        if see_more:
            see_more[0].click()
            time.sleep(1.2)
        driver.execute_script("window.scrollBy(0, arguments[0]);", 1200)
        time.sleep(2.0)
        curr_len = len(urls)
        if curr_len == prev_len:
            stagnant += 1
        else:
            stagnant = 0
        prev_len = curr_len
        loops += 1
        if (reached_old and stagnant >= 2) or stagnant >= 6 or loops >= 3000:
            break
    return urls

def main():
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--lang=en-US")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument(f"--user-data-dir={PROFILE_PATH}")
    options.page_load_strategy = "eager"
    driver = uc.Chrome(options=options, use_subprocess=True)
    driver.set_page_load_timeout(90)
    driver.set_script_timeout(90)
    driver.command_executor._client_config.timeout = 300
    login_to_facebook(driver)
    all_urls = collect_group_posts_last_5y(driver, PAGE_URL)
    driver.quit()
    if all_urls:
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['PostURL'])
            for u in sorted(all_urls):
                w.writerow([u])

if __name__ == "__main__":
    main()

TimeoutException: Message: timeout: Timed out receiving message from renderer: 90.000
  (Session info: chrome=140.0.7339.129)
Stacktrace:
	GetHandleVerifier [0x0xa7c333+65459]
	GetHandleVerifier [0x0xa7c374+65524]
	(No symbol) [0x0x89d973]
	(No symbol) [0x0x88e184]
	(No symbol) [0x0x88deba]
	(No symbol) [0x0x88bdc0]
	(No symbol) [0x0x88c8a7]
	(No symbol) [0x0x8994fe]
	(No symbol) [0x0x8aaec5]
	(No symbol) [0x0x8b0a56]
	(No symbol) [0x0x88cf26]
	(No symbol) [0x0x8aab6e]
	(No symbol) [0x0x8aa921]
	(No symbol) [0x0x92b515]
	(No symbol) [0x0x909bf6]
	(No symbol) [0x0x8db38e]
	(No symbol) [0x0x8dc274]
	GetHandleVerifier [0x0xcfeda3+2697763]
	GetHandleVerifier [0x0xcf9ec7+2677575]
	GetHandleVerifier [0x0xaa4194+228884]
	GetHandleVerifier [0x0xa949f8+165496]
	GetHandleVerifier [0x0xa9b18d+192013]
	GetHandleVerifier [0x0xa847d8+99416]
	GetHandleVerifier [0x0xa84972+99826]
	GetHandleVerifier [0x0xa6ebea+10346]
	BaseThreadInitThunk [0x0x765d5d49+25]
	RtlInitializeExceptionChain [0x0x772cd6db+107

In [6]:
import time
import os
import csv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
INPUT_LINKS_CSV = 'Facebook_group_post_urls_last5y.csv'
OUTPUT_DETAILS_CSV = 'scraped_full_content.csv'
PROFILE_PATH = r'C:\\chrome-profiles\\fb-scraper-profile'
WEBDRIVER_WAIT_TIMEOUT = 20

def get_raw_post_text(driver, permalink):
    wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)
    driver.get(permalink)

    dialog_selector = "div[role='dialog']"
    dialog_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, dialog_selector)))

    see_more_xpath = ".//div[@role='button' and (contains(., 'See more') or contains(., 'เพิ่มเติม'))]"
    
    see_more_buttons = dialog_element.find_elements(By.XPATH, see_more_xpath)
    if see_more_buttons:
        driver.execute_script("arguments[0].click();", see_more_buttons[0])
        time.sleep(1.5)

    content_container_selector = 'div[data-ad-rendering-role="story_message"]'
    container = dialog_element.find_element(By.CSS_SELECTOR, content_container_selector)
    post_text = container.text
    return post_text

if not FACEBOOK_EMAIL or not FACEBOOK_PASSWORD:
    print("Error: Facebook credentials not found. Please check your .env file.")
    exit()

print("Initializing Chrome Driver...")
options = uc.ChromeOptions()
options.add_argument(f'--user-data-dir={PROFILE_PATH}')
options.add_argument('--disable-notifications')
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

print("Logging into Facebook...")
driver.get("https://www.facebook.com")
time.sleep(3)

cookie_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[data-cookiebanner="accept_button"]')
if cookie_buttons and cookie_buttons[0].is_displayed():
    cookie_buttons[0].click()
    time.sleep(1)

if "login" in driver.current_url:
    wait.until(EC.presence_of_element_located((By.ID, "email"))).send_keys(FACEBOOK_EMAIL)
    driver.find_element(By.ID, "pass").send_keys(FACEBOOK_PASSWORD)
    driver.find_element(By.NAME, "login").click()

wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]')))
print("Login successful. Starting scraping process...")

with open(INPUT_LINKS_CSV, 'r', encoding='utf-8') as infile, \
      open(OUTPUT_DETAILS_CSV, 'w', newline='', encoding='utf-8') as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    header = ['Post_URL', 'Full_Post_Content']
    writer.writerow(header)

    next(reader, None)

    for i, row in enumerate(reader):
        if not row:
            continue
        permalink = row[0]
        if not permalink:
            continue

        print(f"Processing link {i+1}: {permalink}")
        
        raw_text = "N/A"
        try:
            raw_text = get_raw_post_text(driver, permalink)
        except Exception as e:
            print(f"  - FAILED to process link. Reason: {e}")
            raw_text = 'ERROR: Main process failed.'

        output_row = [permalink, raw_text]
        writer.writerow(output_row)

        time.sleep(3)

print(f"Scraping complete. Data saved to {OUTPUT_DETAILS_CSV}")
driver.quit()

Initializing Chrome Driver...
Logging into Facebook...
Login successful. Starting scraping process...
Processing link 1: https://www.facebook.com/groups/322977734828852/posts/1725715287888416/
Processing link 2: https://www.facebook.com/groups/322977734828852/posts/1855015104958433/
Processing link 3: https://www.facebook.com/groups/322977734828852/posts/1886307071829236/
Processing link 4: https://www.facebook.com/groups/322977734828852/posts/1976119702847972/
Processing link 5: https://www.facebook.com/groups/322977734828852/posts/2028623960930879/
Processing link 6: https://www.facebook.com/groups/322977734828852/posts/2116491898810751/
Processing link 7: https://www.facebook.com/groups/322977734828852/posts/2120534288406512/
Processing link 8: https://www.facebook.com/groups/322977734828852/posts/2154369618356312/
Processing link 9: https://www.facebook.com/groups/322977734828852/posts/2173414889785118/
Processing link 10: https://www.facebook.com/groups/322977734828852/posts/22072

In [ ]:
import csv
import re

INPUT_CSV = 'scraped_full_content.csv'
OUTPUT_CSV = 'extracted_property_details.csv'

def extract_details(text):
    prop_type = 'N/A'
    property_types = ['บ้านเดี่ยว', 'ทาวน์โฮม', 'ทาวน์เฮ้าส์', 'คอนโด', 'ที่ดิน', 'อาคารพาณิชย์', 'ตึกแถว', 'สวนลำไย']
    for p_type in property_types:
        if p_type in text:
            prop_type = p_type
            break
    if 'บ้านพร้อมที่ดิน' in text:
        prop_type = 'บ้านพร้อมที่ดิน'

    price = 'N/A'
    price_match = re.search(r'([\d,.]+)\s*(ล้าน|บาท|/ด)', text)
    if price_match:
        price = price_match.group(0).strip()
    price_match_2 = re.search(r'(\d{4,})/ด', text)
    if not price_match and price_match_2:
        price = price_match_2.group(0).strip()

    location = 'N/A'
    location_keywords = [
        'หมู่บ้าน', 'โครงการ', 'ถนน', 'ซอย', 'ตำบล', 'อำเภอ', 'ย่าน', 'ติด', 'ใกล้',
        'สันกำแพง', 'สันทราย', 'หางดง', 'สารภี', 'แม่ริม', 'รวมโชค', 'หนองจ๊อม',
        'แม่โจ้', 'จอมทอง', 'ป่าซาง'
    ]
    loc_pattern = r'(' + '|'.join(location_keywords) + r')[\s\w.()-]+'
    loc_match = re.search(loc_pattern, text)
    if loc_match:
        location = loc_match.group(0).strip().replace('\n', ' ')

    size_parts = []
    land_size_matches = re.findall(r'(\d[\d,.]*\s*(?:ไร่|งาน|ตร\.?ว\.?|ตารางวา))', text)
    if land_size_matches:
        size_parts.extend(land_size_matches)
    beds_match = re.search(r'(\d+)\s*(?:ห้อง)?นอน', text)
    if beds_match:
        size_parts.append(f"{beds_match.group(1)} นอน")
    baths_match = re.search(r'(\d+)\s*(?:ห้อง)?น้ำ', text)
    if baths_match:
        size_parts.append(f"{baths_match.group(1)} น้ำ")
        
    size = ' '.join(size_parts) if size_parts else 'N/A'

    return {
        'ประเภท': prop_type,
        'ราคา': price,
        'ทำเล': location,
        'ขนาด': size,
    }

if __name__ == "__main__":
    with open(INPUT_CSV, mode='r', encoding='utf-8') as infile, \
         open(OUTPUT_CSV, mode='w', encoding='utf-8', newline='') as outfile:
        
        reader = csv.DictReader(infile)
        
        fieldnames = ['ประเภท', 'ราคา', 'ทำเล', 'ขนาด', 'Link']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            post_url = row.get('Post_URL', '')
            full_content = row.get('Full_Post_Content', '')

            details = extract_details(full_content)
            
            output_row = {
                'ประเภท': details['ประเภท'],
                'ราคา': details['ราคา'],
                'ทำเล': details['ทำเล'],
                'ขนาด': details['ขนาด'],
                'Link': post_url
            }
            writer.writerow(output_row)

In [ ]:
#DDproperty Scraper
import time
import csv
import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

START_URL = 'https://www.ddproperty.com/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2?areaCode=&listingType=sale&regionCode=TH50&locale=th&slug=search&_freetextDisplay=%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88&isCommercial=false'
OUTPUT_CSV_FILE = 'ddproperty_listing_urls.csv'
WEBDRIVER_WAIT_TIMEOUT = 20
MAX_PAGES = 200

def build_page_url(start_url, page):
    u = urlparse(start_url)
    q = urlencode(dict(parse_qsl(u.query)), doseq=True)
    path = u.path.rstrip('/')
    m = re.match(r'^(.*?)(/\d+)$', path)
    if m:
        base = m.group(1)
    else:
        base = path
    if page <= 1:
        new_path = base
    else:
        new_path = base + f'/{page}'
    return urlunparse((u.scheme, u.netloc, new_path, u.params, q, u.fragment))

def main():
    print("Initializing Chrome Driver...")
    options = uc.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--start-maximized')
    driver = uc.Chrome(options=options)
    wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

    first_url = build_page_url(START_URL, 1)
    print(f"Navigating to start URL: {first_url}")
    driver.get(first_url)

    btns = driver.find_elements(By.XPATH, "//button[normalize-space(text())='ยอมรับ']")
    if btns:
        btns[0].click()
        time.sleep(1)

    all_listing_urls = set()
    page_num = 1
    last_first_href = ""

    while page_num <= MAX_PAGES:
        print(f"\n--- Scraping Page {page_num} ---")
        listing_sel = "a.listing-card-link"
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, listing_sel)))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.4)
        cards = driver.find_elements(By.CSS_SELECTOR, listing_sel)
        hrefs_now = [e.get_attribute('href') for e in cards if e.get_attribute('href')]
        if not hrefs_now:
            print("No URLs found on this page, stopping.")
            break
        if page_num > 1 and hrefs_now[0] == last_first_href:
            print("Listing did not change from previous page. Ending pagination.")
            break
        last_first_href = hrefs_now[0]
        new_urls = set(hrefs_now) - all_listing_urls
        print(f"Found {len(new_urls)} new listing URLs on this page.")
        all_listing_urls.update(hrefs_now)

        next_num = page_num + 1
        next_url = build_page_url(START_URL, next_num)
        prev_last = hrefs_now[-1]
        driver.get(next_url)

        t0 = time.time()
        changed = False
        while time.time() - t0 < WEBDRIVER_WAIT_TIMEOUT:
            state = driver.execute_script("return document.readyState")
            if state == "complete":
                els = driver.find_elements(By.CSS_SELECTOR, listing_sel)
                if els:
                    h = els[0].get_attribute('href')
                    hh = els[-1].get_attribute('href')
                    if h and h != last_first_href:
                        changed = True
                        break
                    if hh and hh != prev_last:
                        changed = True
                        break
            time.sleep(0.3)
        if not changed:
            print("Next page did not change within timeout. Ending pagination.")
            break

        page_num = next_num

    driver.quit()

    if all_listing_urls:
        print(f"\nTotal unique URLs collected: {len(all_listing_urls)}")
        print(f"Writing all URLs to {OUTPUT_CSV_FILE}...")
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['ListingURL'])
            for url in sorted(all_listing_urls):
                w.writerow([url])
        print("Process complete.")
    else:
        print("No URLs were collected.")

if __name__ == "__main__":
    main()


Initializing Chrome Driver...
Navigating to start URL: https://www.ddproperty.com/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2?listingType=sale&regionCode=TH50&locale=th&slug=search&_freetextDisplay=%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88&isCommercial=false

--- Scraping Page 1 ---
Found 25 new listing URLs on this page.

--- Scraping Page 2 ---
Found 20 new listing URLs on this page.

--- Scraping Page 3 ---
Found 20 new listing URLs on this page.

--- Scraping Page 4 ---
Found 20 new listing URLs on this page.

--- Scraping Page 5 ---
Found 20 new listing URLs on this page.

--- Scraping Page 6 ---
Found 20 new listing URLs on this page.

--- Scraping Page 7 ---
Found 20 new listing URLs on this page.

--- Scraping Page 8 ---
Found 20 new listing URLs on this page.

--- Scraping Page 9 ---
Found 20 new listing URLs on this page.

--- Scraping Page 10 ---
Found 20 new listing URLs

In [1]:
import time
import os
import csv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- CONFIGURATION ---
INPUT_CSV_FILE = 'ddproperty_listing_urls.csv'
OUTPUT_CSV_FILE = 'ddproperty_scraped_details.csv'
WEBDRIVER_WAIT_TIMEOUT = 25

def scrape_listing_details(driver, url):
    """
    Scrapes all required details from a single property listing URL.
    """
    wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)
    details = {'URL': url}

    driver.get(url)

    # --- Wait for the main content to bypass Cloudflare and load ---
    try:
        main_content_selector = "div[da-id='property-snapshot-info']"
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, main_content_selector)))
        print("  - Page loaded successfully.")
    except Exception as e:
        print(f"  - FAILED to load main content. Cloudflare might be active. Reason: {e}")
        return {**details, 'Title': 'LOAD_FAILED', 'Price': 'N/A'}

    # --- 1. Scrape Basic Snapshot Info ---
    try:
        details['Title'] = driver.find_element(By.CSS_SELECTOR, "h1[da-id='property-title']").text
        details['Address'] = driver.find_element(By.CSS_SELECTOR, "p[da-id='property-address']").text
        details['Price'] = driver.find_element(By.CSS_SELECTOR, "h2[da-id='price-amount']").text.replace('฿', '').strip()
        
        amenities = driver.find_elements(By.CSS_SELECTOR, "div.amenity[da-id*='-amenity']")
        for amenity in amenities:
            label = amenity.find_element(By.TAG_NAME, 'img').get_attribute('aria-label')
            value = amenity.find_element(By.CSS_SELECTOR, "p.amenity-text").text
            if label and value:
                details[label] = value

    except Exception as e:
        print(f"  - Warning: Could not scrape basic info. {e}")

    # --- 2. Handle "Property Details" Modal ---
    try:
        see_details_btn = driver.find_element(By.CSS_SELECTOR, "button[da-id='meta-table-see-more-btn']")
        driver.execute_script("arguments[0].click();", see_details_btn)
        
        modal_content_selector = "div.property-modal-body"
        modal_body = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, modal_content_selector)))
        
        wrappers = modal_body.find_elements(By.CSS_SELECTOR, "div.property-modal-body-wrapper")
        for wrapper in wrappers:
            icon_alt = wrapper.find_element(By.TAG_NAME, 'img').get_attribute('alt')
            value = wrapper.find_element(By.CSS_SELECTOR, "p.property-modal-body-value").text
            if icon_alt and value:
                key = icon_alt.replace('-o', '').replace('-', ' ').title()
                details[key] = value
        
        close_btn = driver.find_element(By.CSS_SELECTOR, "button[da-id='modal-close-button']")
        driver.execute_script("arguments[0].click();", close_btn)
        wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, modal_content_selector)))

    except Exception:
        pass

    # --- 3. Handle "Description" Modal ---
    try:
        read_more_btn = driver.find_element(By.CSS_SELECTOR, "button[da-id='description-widget-show-more-lnk']")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", read_more_btn)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", read_more_btn)

        desc_modal_selector = "div.description-modal-body"
        desc_body = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, desc_modal_selector)))
        
        details['Description'] = desc_body.get_attribute('innerHTML').replace('<br>', '\n').strip()

        close_btn = driver.find_element(By.CSS_SELECTOR, "div.description-modal-body + button.btn-close")
        driver.execute_script("arguments[0].click();", close_btn)
        wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, desc_modal_selector)))

    except Exception:
        try:
            desc_container = driver.find_element(By.CSS_SELECTOR, "div[da-id='property-description-body-content']")
            details['Description'] = desc_container.text
        except Exception:
            details['Description'] = 'N/A'
            
    return details

def main():
    print("Initializing Chrome Driver...")
    options = uc.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--start-maximized')
    
    # --- FIXED ---
    # Explicitly specify the main Chrome version to match the installed browser.
    # The error log shows your browser is version 140.
    driver = uc.Chrome(options=options, version_main=140)

    # Read all URLs from the input CSV
    try:
        with open(INPUT_CSV_FILE, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader) # Skip header
            urls_to_scrape = [row[0] for row in reader if row]
    except FileNotFoundError:
        print(f"Error: Input file not found at '{INPUT_CSV_FILE}'")
        driver.quit()
        return

    print(f"Found {len(urls_to_scrape)} URLs to process.")
    
    all_details = []
    
    # --- LOOP THROUGH EACH URL ---
    for i, url in enumerate(urls_to_scrape):
        print(f"\nProcessing URL {i+1}/{len(urls_to_scrape)}: {url}")
        details = scrape_listing_details(driver, url)
        all_details.append(details)
        time.sleep(3) # Be polite to the server

    driver.quit()

    # --- WRITING TO CSV ---
    if all_details:
        print(f"\nScraping complete. Writing {len(all_details)} records to {OUTPUT_CSV_FILE}...")
        
        # Get all unique headers from the scraped data
        headers = sorted(list(set(key for d in all_details for key in d.keys())))
        
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=headers)
            writer.writeheader()
            writer.writerows(all_details)
        
        print("Process complete.")

if __name__ == "__main__":
    main()

Initializing Chrome Driver...
Found 2227 URLs to process.

Processing URL 1/2227: https://www.ddproperty.com/property/%E0%B8%81%E0%B8%A3%E0%B8%B5%E0%B8%99%E0%B8%A7%E0%B8%B4%E0%B8%A5%E0%B8%A5%E0%B9%8C%E0%B9%82%E0%B8%AE%E0%B8%A1-%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88-%E0%B8%82%E0%B8%B2%E0%B8%A2-11551760
  - Page loaded successfully.

Processing URL 2/2227: https://www.ddproperty.com/property/%E0%B8%81%E0%B8%AD%E0%B8%9B%E0%B8%A3%E0%B8%AA%E0%B8%B8%E0%B8%82-%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88-%E0%B8%82%E0%B8%B2%E0%B8%A2-11232951
  - Page loaded successfully.

Processing URL 3/2227: https://www.ddproperty.com/property/%E0%B8%81%E0%B8%AD%E0%B8%9B%E0%B8%A3%E0%B8%AA%E0%B8%B8%E0%B8%82-%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88-%E0%B8%82%E0%B8%B2%E0%B8%A2-6846162
  - Page loaded successfully.

Processing URL 4/2227: https://www.ddproperty.com/property/%E0%B8%81%E0%B8%AD%

In [ ]:
#LivingInsider Scraper

import time
import csv
import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

START_URL = 'https://www.livinginsider.com/living_zone/45/all/all/1/%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88.html'
OUTPUT_CSV_FILE = 'livinginsider_listing_urls.csv'
PAGE_TIMEOUT = 45
MAX_PAGES = 200

def build_page_url(u, page):
    p = urlparse(u)
    parts = [x for x in p.path.split('/') if x]
    k = None
    for i, seg in enumerate(parts):
        if re.fullmatch(r'\d+', seg):
            k = i
    if k is None:
        base = p.path.rstrip('/')
        new_path = base if page <= 1 else base + f'/{page}'
    else:
        parts[k] = '1' if page <= 1 else str(page)
        new_path = '/' + '/'.join(parts)
    return urlunparse((p.scheme, p.netloc, new_path, p.params, urlencode(dict(parse_qsl(p.query)), doseq=True), p.fragment))

def wait_ready(driver, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.execute_script("return document.readyState") == "complete":
            return True
        time.sleep(0.2)
    return False

def deep_scroll(driver, rounds=18, pause=0.7):
    h0 = 0
    for _ in range(rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        h = driver.execute_script("return document.body.scrollHeight")
        if h == h0:
            break
        h0 = h

def collect_links_js(driver, base):
    js = """
const out = [];
const a1 = Array.from(document.querySelectorAll("a[href*='/livingdetail/'][href$='.html']"));
const a2 = Array.from(document.querySelectorAll("a[href^='/livingdetail/'][href$='.html']"));
const all = [...a1, ...a2];
for (const a of all){
  const h = a.getAttribute('href') || '';
  if (h && !h.includes('bclick') && !h.includes('stories') && !h.includes('banner')){
    out.push(h);
  }
}
return Array.from(new Set(out));
"""
    hrefs = driver.execute_script(js)
    fixed = []
    for h in hrefs:
        if h.startswith("/"):
            fixed.append(base + h)
        else:
            fixed.append(h)
    return list(dict.fromkeys(fixed))

def main():
    options = uc.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--start-maximized')
    driver = uc.Chrome(options=options)

    base = f"{urlparse(START_URL).scheme}://{urlparse(START_URL).netloc}"

    page = 1
    all_urls = set()
    last_first = ""
    driver.get(build_page_url(START_URL, page))
    wait_ready(driver, PAGE_TIMEOUT)

    while page <= MAX_PAGES:
        deep_scroll(driver, rounds=20, pause=0.8)
        urls_now = collect_links_js(driver, base)
        if not urls_now:
            break
        if page > 1 and urls_now[0] == last_first:
            break
        last_first = urls_now[0]
        for u in urls_now:
            all_urls.add(u)

        page += 1
        prev_last = urls_now[-1]
        driver.get(build_page_url(START_URL, page))
        t0 = time.time()
        changed = False
        while time.time() - t0 < PAGE_TIMEOUT:
            wait_ready(driver, 3)
            deep_scroll(driver, rounds=6, pause=0.6)
            cur = collect_links_js(driver, base)
            if cur:
                if cur[0] != last_first or cur[-1] != prev_last:
                    changed = True
                    break
            time.sleep(0.3)
        if not changed:
            break

    driver.quit()

    if all_urls:
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['ListingURL'])
            for u in sorted(all_urls):
                w.writerow([u])

if __name__ == "__main__":
    main()


In [5]:
import re
import csv
import sys
import time
from pathlib import Path
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

INPUT_CSV = Path("Scraping") / "livinginsider_listing_urls.csv"
OUTPUT_CSV = "livinginsider_scraped_details.csv"
WAIT = 30

def first_text(elems):
    return elems[0].text.strip() if elems else ""

def resolve_csv(p):
    if p.exists():
        return p
    hits = [c for c in Path.cwd().rglob("livinginsider_listing_urls.csv") if c.parent.name == "Scraping"]
    if hits:
        return hits[0]
    print("CSV not found at Scraping\\livinginsider_listing_urls.csv")
    sys.exit(1)

def click_consent(driver):
    btns = []
    btns += driver.find_elements(By.CSS_SELECTOR, "#onetrust-accept-btn-handler")
    btns += driver.find_elements(By.XPATH, "//*[self::a or self::button][contains(., 'ยอมรับ')]")
    btns = [e for e in btns if e.is_displayed() and e.is_enabled()]
    if btns:
        driver.execute_script("arguments[0].click();", btns[0])
        time.sleep(0.3)

def wait_present(driver, by, sel, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.find_elements(by, sel):
            return True
        time.sleep(0.25)
    return False

def page_loaded(driver):
    try_len = driver.execute_script("return document.body && document.body.innerHTML ? document.body.innerHTML.length : 0")
    return try_len and try_len > 1000

def human_reload_flow(driver, url):
    driver.get(url)
    click_consent(driver)
    ok = wait_present(driver, By.CSS_SELECTOR, "div.space_padding_top_data", 8)
    if ok or page_loaded(driver):
        return True
    driver.refresh()
    click_consent(driver)
    ok = wait_present(driver, By.CSS_SELECTOR, "div.space_padding_top_data", 8)
    if ok or page_loaded(driver):
        return True
    driver.get("https://www.livinginsider.com/")
    time.sleep(1.2)
    driver.get(url)
    click_consent(driver)
    ok = wait_present(driver, By.CSS_SELECTOR, "div.space_padding_top_data", 8)
    if ok or page_loaded(driver):
        return True
    return False

def parse_coords(text):
    m = re.search(r"(-?\d+(?:\.\d+)?)\s*,\s*(-?\d+(?:\.\d+)?)", text)
    return f"{m.group(1)},{m.group(2)}" if m else ""

def scrape_one(driver, url):
    ok = human_reload_flow(driver, url)
    if not ok:
        return None
    title = first_text(driver.find_elements(By.CSS_SELECTOR, "h1.font_sarabun.show-title"))
    price = first_text(driver.find_elements(By.CSS_SELECTOR, ".show_price_topic .price_topic b"))
    price_per_area = first_text(driver.find_elements(By.CSS_SELECTOR, ".show_price_topic .price_cal_area_text"))
    prop_type = first_text(driver.find_elements(By.CSS_SELECTOR, ".box_tag_topic_detail .box_tag_building"))
    post_type = first_text(driver.find_elements(By.CSS_SELECTOR, ".box_tag_topic_detail .box_tag_posttype"))
    created_date = first_text(driver.find_elements(By.CSS_SELECTOR, ".row-detail-time .font_10_date"))
    boosted_text = first_text(driver.find_elements(By.XPATH, "//div[contains(@class,'row-detail-time')]//span[contains(text(),'ดันประกาศล่าสุด')]"))
    size_text = first_text(driver.find_elements(By.CSS_SELECTOR, ".detail_property_list_des_new .detail-property-list-text"))
    description = first_text(driver.find_elements(By.CSS_SELECTOR, "#desc-text-nl .wordwrap-box .wordwrap"))
    location_text = first_text(driver.find_elements(By.CSS_SELECTOR, ".form-group.group-location-detail .detail-text-zone a"))
    vc = [e.text.strip() for e in driver.find_elements(By.CSS_SELECTOR, ".box-show-view-click .text-custom-gray-new")]
    views = vc[0] if len(vc) > 0 else ""
    clicks = vc[1] if len(vc) > 1 else ""
    coords = parse_coords(description)
    phone_nodes = driver.find_elements(By.CSS_SELECTOR, "a.p-phone-contact[data-zcgrbcb]")
    phones_masked = []
    for n in phone_nodes:
        v = n.get_attribute("data-zcgrbcb") or ""
        if v:
            phones_masked.append(v)
    phones_masked = ", ".join(sorted(set(phones_masked)))
    return {
        "URL": url,
        "Title": title,
        "Price": price,
        "PricePerArea": price_per_area,
        "PropertyType": prop_type,
        "PostType": post_type,
        "Size": size_text,
        "Description": description,
        "CreatedDate": created_date,
        "Boosted": boosted_text,
        "Location": location_text,
        "Views": views,
        "Clicks": clicks,
        "Coordinates": coords,
        "MaskedContacts": phones_masked
    }

csv_path = resolve_csv(INPUT_CSV)
options = uc.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options, version_main=140)

with open(str(csv_path), "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    next(reader, None)
    urls = [r[0].strip() for r in reader if r and r[0].strip()]

rows = []
for u in urls:
    r = scrape_one(driver, u)
    if r:
        rows.append(r)
    time.sleep(0.4)

driver.quit()

with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=[
        "URL","Title","Price","PricePerArea","PropertyType","PostType",
        "Size","Description","CreatedDate","Boosted","Location",
        "Views","Clicks","Coordinates","MaskedContacts"
    ])
    w.writeheader()
    w.writerows(rows)

In [ ]:
#Kaidee Scraper

import time
import csv
import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

START_URL = "https://baan.kaidee.com/c2p1-realestate/chiangmai"
OUTPUT_CSV_FILE = "kaidee_listing_urls.csv"
PAGE_TIMEOUT = 40
MAX_PAGES = 200

PATTERN = re.compile(r"https://baan\.kaidee\.com/product-\d+")

def build_page_url(u, page):
    if page <= 1:
        return u
    if u.endswith("/"):
        return f"{u}p-{page}"
    return f"{u}/p-{page}"

def wait_ready(driver, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.execute_script("return document.readyState") == "complete":
            return True
        time.sleep(0.2)
    return False

def deep_scroll(driver, rounds=12, pause=0.8):
    prev = 0
    for _ in range(rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        cur = driver.execute_script("return document.body.scrollHeight")
        if cur == prev:
            break
        prev = cur

def extract_links(html):
    urls = set(PATTERN.findall(html))
    return sorted(urls)

def main():
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")
    driver = uc.Chrome(options=options)

    page = 1
    all_urls = set()
    last_first = ""

    while page <= MAX_PAGES:
        url = build_page_url(START_URL, page)
        driver.get(url)
        wait_ready(driver, PAGE_TIMEOUT)
        deep_scroll(driver, rounds=15, pause=0.8)

        html = driver.page_source
        links = extract_links(html)

        if not links:
            break
        if page > 1 and links[0] == last_first:
            break

        last_first = links[0]
        all_urls.update(links)
        page += 1

    driver.quit()

    if all_urls:
        with open(OUTPUT_CSV_FILE, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["ListingURL"])
            for u in sorted(all_urls):
                w.writerow([u])

if __name__ == "__main__":
    main()


In [14]:
import csv
import time
from pathlib import Path
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

OUTPUT_CSV_FILE = "kaidee_scraped_details.csv"
WAIT = 25

def resolve_input_csv():
    cands = [
        Path("Scraping") / "kaidee_listing_urls.csv",
        Path("kaidee_listing_urls.csv")
    ]
    for p in cands:
        if p.exists():
            return p
    found = list(Path.cwd().rglob("kaidee_listing_urls.csv"))
    if found:
        return found[0]
    print("CSV not found. Checked:", ", ".join(str(x.resolve()) for x in cands))
    print("Also searched recursively under:", str(Path.cwd().resolve()))
    return None

def scrape(driver, url):
    w = WebDriverWait(driver, WAIT)
    driver.get(url)
    btns = driver.find_elements(By.CSS_SELECTOR, "button[aria-label*='cookie i understand'], button[aria-label*='accept'], button:has(span[lang])")
    if btns:
        driver.execute_script("arguments[0].click();", btns[0])
        time.sleep(0.3)
    title_el = w.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.sc-747m9u-7")))
    price_el = w.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-1w68tq4-0 span.sc-3tpgds-0")))
    desc_el = driver.find_elements(By.CSS_SELECTOR, "div.sc-1kndlp1-0 p.inner-text")
    area_el = driver.find_elements(By.XPATH, "//ul[@id='has-attributes']//li[.//span[text()='เนื้อที่']]//span//b")
    phone_els = driver.find_elements(By.CSS_SELECTOR, "span.masked[data-value]")
    title = title_el.text.strip()
    price = price_el.text.strip()
    desc = desc_el[0].text.strip() if desc_el else ""
    area = area_el[0].text.strip() if area_el else ""
    phones = sorted({e.get_attribute("data-value").strip() for e in phone_els if e.get_attribute("data-value")})
    return {"URL": url, "Title": title, "Price": price, "Area": area, "Phone": ", ".join(phones), "Description": desc}

def main():
    input_csv = resolve_input_csv()
    if not input_csv:
        return
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")
    driver = uc.Chrome(options=options, version_main=140)
    with open(input_csv, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader, None)
        urls = [r[0].strip() for r in reader if r and r[0].strip()]
    rows = []
    for u in urls:
        rows.append(scrape(driver, u))
        time.sleep(1)
    driver.quit()
    with open(OUTPUT_CSV_FILE, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["URL", "Title", "Price", "Area", "Phone", "Description"])
        w.writeheader()
        w.writerows(rows)

if __name__ == "__main__":
    main()


In [3]:
import time
import os
import csv
from dotenv import load_dotenv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- CONFIGURATION ---
load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
START_URL = 'https://www.facebook.com/marketplace/chiangmai/propertyrentals/?exact=false'
OUTPUT_CSV_FILE = 'facebook_marketplace_urls.csv'
TARGET_URL_COUNT = 500
WEBDRIVER_WAIT_TIMEOUT = 20

def handle_login_popup(driver, wait):
    """
    Detects and handles the specific Marketplace login pop-up.
    """
    print("Checking for login pop-up...")
    try:
        # Wait for the unique container of the login pop-up
        popup_container_selector = "div[aria-label='Accessible login form']"
        popup_form = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, popup_container_selector)))
        
        print("Login pop-up detected. Attempting to log in...")
        
        email_input = popup_form.find_element(By.NAME, "email")
        pass_input = popup_form.find_element(By.NAME, "pass")
        
        # --- CRITICAL FIX: Using the correct aria-label for the login button ---
        login_button_selector = "div[aria-label='Log in to Facebook']"
        login_button = popup_form.find_element(By.CSS_SELECTOR, login_button_selector)

        email_input.send_keys(FACEBOOK_EMAIL)
        pass_input.send_keys(FACEBOOK_PASSWORD)
        login_button.click()

        print("Login submitted. Waiting for pop-up to disappear...")
        wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, popup_container_selector)))
        print("Login successful.")
        time.sleep(5) # Extra wait for main content to settle after login

    except Exception:
        # This will trigger if the pop-up is not found within the timeout, which is expected if already logged in.
        print("No login pop-up detected. Assuming already logged in.")
        pass

# --- SCRIPT EXECUTION STARTS HERE ---

print("Initializing Chrome Driver...")
options = uc.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--start-maximized')
options.add_argument(f'--user-data-dir=C:/chrome-profiles/fb-marketplace')

driver = uc.Chrome(options=options, version_main=140)
wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

driver.get(START_URL)

# Call the robust login handler
handle_login_popup(driver, wait)

print("Login check complete. Starting scroll loop...")
all_listing_urls = set()
stagnant_scrolls = 0

while len(all_listing_urls) < TARGET_URL_COUNT:
    print(f"\rCollected {len(all_listing_urls)} / {TARGET_URL_COUNT} URLs...", end="")
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    item_link_selector = "a[href*='/marketplace/item/']"
    try:
        # Use a longer wait here to ensure content loads after scrolling
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, item_link_selector)))
    except Exception:
        print("\nCould not find any item links. The page might be empty or structure has changed.")
        break

    listing_elements = driver.find_elements(By.CSS_SELECTOR, item_link_selector)
    
    for element in listing_elements:
        url = element.get_attribute('href')
        if url:
            full_url = f"https://www.facebook.com{url.split('?')[0]}"
            all_listing_urls.add(full_url)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5) # Generous wait for new content to load

    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        stagnant_scrolls += 1
    else:
        stagnant_scrolls = 0

    if stagnant_scrolls >= 5:
        print(f"\nPage height has not changed for {stagnant_scrolls} consecutive scrolls. Assuming end of content.")
        break

driver.quit()

if all_listing_urls:
    final_urls = sorted(list(all_listing_urls))[:TARGET_URL_COUNT]
    print(f"\nTotal unique URLs collected: {len(final_urls)}")
    print(f"Writing URLs to {OUTPUT_CSV_FILE}...")
    
    with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['MarketplaceURL'])
        for url in final_urls:
            writer.writerow([url])
    
    print("Process complete.")
else:
    print("No URLs were collected.")

Initializing Chrome Driver...
Checking for login pop-up...
No login pop-up detected. Assuming already logged in.
Login check complete. Starting scroll loop...
Collected 494 / 500 URLs...
Total unique URLs collected: 500
Writing URLs to facebook_marketplace_urls.csv...
Process complete.


In [9]:
import time
import csv
from pathlib import Path
from dotenv import load_dotenv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException

# ------------------------------------
# ENV
# ------------------------------------
load_dotenv()
from os import getenv
FACEBOOK_EMAIL = getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = getenv('FACEBOOK_PASSWORD')
assert FACEBOOK_EMAIL and FACEBOOK_PASSWORD, "Missing FACEBOOK_EMAIL or FACEBOOK_PASSWORD in .env"

# ------------------------------------
# PATHS (ใช้ absolute path ตรง ๆ ไม่ผ่าน os.path)
# ------------------------------------
INPUT_CSV_FILE  = Path(r"C:\Users\kongl\Documents\GitHub\Real-Estate Listing Aggregator System\Scraping\facebook_marketplace_urls.csv")
OUTPUT_CSV_FILE = Path(r"C:\Users\kongl\Documents\GitHub\Real-Estate Listing Aggregator System\Scraping\facebook_marketplace_details.csv")
assert INPUT_CSV_FILE.exists(), f"INPUT_CSV_FILE not found: {INPUT_CSV_FILE}"

# ------------------------------------
# CONFIG
# ------------------------------------
WEBDRIVER_WAIT_TIMEOUT = 25
SCROLL_STEP_PX = 900
MAX_SCROLL_TRIES = 15

# ------------------------------------
# HELPERS
# ------------------------------------
def safe_text(el):
    try:
        return el.text.strip()
    except Exception:
        return ""

def handle_login_popup(driver, wait):
    """
    จัดการป็อปอัปล็อกอินทั้ง EN/TH ถ้าไม่มีให้ผ่านไป
    """
    time.sleep(7)
    try:
        popup_container_selector = (
            "div[aria-label='Accessible login form'], "
            "div[aria-label='เข้าสู่ระบบแบบช่วยการเข้าถึง']"
        )
        popup_forms = driver.find_elements(By.CSS_SELECTOR, popup_container_selector)
        if popup_forms:
            popup_form = popup_forms[0]
            email_input = popup_form.find_element(By.NAME, "email")
            pass_input  = popup_form.find_element(By.NAME, "pass")
            # ปุ่มทั้ง EN/TH
            login_button = popup_form.find_element(
                By.CSS_SELECTOR,
                "div[aria-label*='Log in'], div[aria-label*='เข้าสู่ระบบ']"
            )

            email_input.clear(); email_input.send_keys(FACEBOOK_EMAIL)
            pass_input.clear();  pass_input.send_keys(FACEBOOK_PASSWORD)
            login_button.click()

            # รอจนป็อปอัปหาย
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, popup_container_selector)))
            time.sleep(5)
    except Exception:
        pass

def progressive_scroll_to_description(driver, wait):
    """
    เลื่อนหน้าจอลงทีละสเต็ป จนกว่าจะเจอหัวข้อ 'คำอธิบาย' หรือ 'Description'
    คืนค่า: element <h2> ของหัวข้อคำอธิบายถ้าพบ, ถ้าไม่พบคืน None
    """
    desc_h2 = None
    last_height = 0

    # ลองหาก่อนแบบไม่เลื่อนมาก
    try:
        desc_h2 = driver.find_element(By.XPATH,
            "//h2[.//span[contains(., 'คำอธิบาย')] or .//span[contains(., 'Description')]]"
        )
        driver.execute_script("arguments[0].scrollIntoView({behavior:'instant', block:'center'});", desc_h2)
        time.sleep(0.5)
        return desc_h2
    except NoSuchElementException:
        pass

    for i in range(MAX_SCROLL_TRIES):
        driver.execute_script(f"window.scrollBy(0, {SCROLL_STEP_PX});")
        time.sleep(0.8)

        # พยายามหาอีกครั้ง
        try:
            desc_h2 = driver.find_element(By.XPATH,
                "//h2[.//span[contains(., 'คำอธิบาย')] or .//span[contains(., 'Description')]]"
            )
            driver.execute_script("arguments[0].scrollIntoView({behavior:'instant', block:'center'});", desc_h2)
            time.sleep(0.6)
            return desc_h2
        except NoSuchElementException:
            pass

        # ถ้าถึงท้ายหน้าแล้วและความสูงไม่เปลี่ยน แสดงว่าน่าจะสุด
        new_height = driver.execute_script("return document.body.scrollHeight;")
        if new_height == last_height:
            # ลองเลื่อนขึ้นเล็กน้อยแล้วลงใหม่ (บางทีคอนเทนต์อยู่กลาง ๆ)
            driver.execute_script("window.scrollTo(0, Math.max(document.documentElement.scrollTop - 600, 0));")
            time.sleep(0.5)
        last_height = new_height

    return None

def click_see_more_if_present(driver, context_el=None):
    """
    คลิกปุ่ม 'ดูเพิ่มเติม' หรือ 'See more' ภายในบริบท (context_el) ถ้าให้มา
    ถ้าไม่ให้มา จะหาทั่วหน้า
    """
    XPATHS = [
        ".//div[@role='button'][.//span[contains(., 'ดูเพิ่มเติม')]]",
        ".//div[@role='button'][.//span[contains(., 'See more')]]",
        "//div[@role='button'][.//span[contains(., 'ดูเพิ่มเติม')]]",
        "//div[@role='button'][.//span[contains(., 'See more')]]",
    ]
    for xp in XPATHS:
        try:
            el = (context_el or driver).find_element(By.XPATH, xp)
            # บางกรณีถ้าขยายแล้วจะเป็น 'ดูน้อยลง/See less' ก็ข้าม
            txt = safe_text(el)
            if "ดูน้อยลง" in txt or "See less" in txt:
                return True
            driver.execute_script("arguments[0].scrollIntoView({behavior:'instant', block:'center'});", el)
            time.sleep(0.2)
            el.click()
            time.sleep(0.6)
            return True
        except (NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException):
            continue
    return False

def extract_full_description(driver, desc_h2):
    """
    รับ h2 ของหัวข้อคำอธิบาย แล้วหา sibling/content ที่เป็นข้อความคำอธิบายจริง
    - พยายามคลิก 'ดูเพิ่มเติม/See more' ก่อนอ่าน
    - คืนข้อความคำอธิบายเต็ม
    """
    description_text = ""

    # กล่องหลักของหัวข้อ
    parent_div = None
    try:
        parent_div = desc_h2.find_element(By.XPATH, "./..")
    except Exception:
        pass

    # คลิก 'ดูเพิ่มเติม' ในบริบทก่อน
    try:
        click_see_more_if_present(driver, parent_div)
    except Exception:
        pass

    # 1) ความพยายามหลัก: sibling div -> span
    XPATH_CANDIDATES = [
        "./following-sibling::div//span[string-length(normalize-space())>0]",
        "./following::div[1]//span[string-length(normalize-space())>0]",
    ]
    for rel_xp in XPATH_CANDIDATES:
        try:
            container = parent_div if parent_div else desc_h2
            spans = container.find_elements(By.XPATH, rel_xp)
            # รวมข้อความทุก span ที่มองเห็น
            parts = [safe_text(s) for s in spans if safe_text(s)]
            if parts:
                description_text = "\n".join(parts)
                break
        except Exception:
            continue

    # 2) Fallback: หา span ยาว ๆ ใกล้ ๆ h2 ทั้งหน้า (กรณีแปลก)
    if not description_text:
        try:
            spans = driver.find_elements(
                By.XPATH,
                "//h2[.//span[contains(., 'คำอธิบาย')] or .//span[contains(., 'Description')]]"
                "/ancestor::div[1]/following-sibling::div//span[string-length(normalize-space())>0]"
            )
            parts = [safe_text(s) for s in spans if safe_text(s)]
            description_text = "\n".join(parts)
        except Exception:
            description_text = ""

    # ตัด duplicated lines ที่ FB ชอบซ้ำ (optional)
    if description_text:
        lines = [ln.rstrip() for ln in description_text.splitlines()]
        # ลบบรรทัดว่างซ้ำ ๆ ข้างเคียง
        cleaned = []
        for ln in lines:
            if not (ln == "" and (len(cleaned) > 0 and cleaned[-1] == "")):
                cleaned.append(ln)
        description_text = "\n".join(cleaned).strip()

    return description_text

def scrape_marketplace_details(driver, url):
    wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)
    details = {'URL': url}

    driver.get(url)

    # รอ title ของโพสต์
    h1 = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))
    details['Title'] = safe_text(h1)

    # ราคา (มี fallback)
    try:
        price_xpath = "//h1/following-sibling::div[1]//span"
        details['Price'] = safe_text(driver.find_element(By.XPATH, price_xpath))
    except Exception:
        try:
            details['Price'] = safe_text(driver.find_element(By.XPATH, "//span[contains(.,'฿') or contains(., '$')]"))
        except Exception:
            details['Price'] = ""

    # รายละเอียด property (listitem)
    try:
        detail_items = driver.find_elements(By.CSS_SELECTOR, "div[role='listitem'] span")
        property_details = [safe_text(item) for item in detail_items if safe_text(item)]
        details['Property_Details'] = ' | '.join(property_details)
    except Exception:
        details['Property_Details'] = ""

    # --- NEW: เลื่อนหา 'คำอธิบาย' + กด 'ดูเพิ่มเติม' + เก็บข้อความเต็ม ---
    desc_text = ""
    desc_h2 = progressive_scroll_to_description(driver, wait)
    if desc_h2:
        # พยายามคลิก 'ดูเพิ่มเติม/See more' ถ้ามี แล้วดึงข้อความเต็ม
        try:
            # เผื่อฟีดมีปุ่มอยู่ด้านล่างข้อความ
            click_see_more_if_present(driver, context_el=desc_h2.find_element(By.XPATH, "./.."))
        except Exception:
            pass
        desc_text = extract_full_description(driver, desc_h2)
    else:
        # หาแบบ quick-fallback ไม่เลื่อน (เผื่อบางหน้ามีอยู่แล้ว)
        try:
            desc_h2 = driver.find_element(By.XPATH, "//h2[.//span[contains(., 'Description') or contains(., 'คำอธิบาย')]]")
            desc_text = extract_full_description(driver, desc_h2)
        except Exception:
            desc_text = ""

    details['Description'] = desc_text

    return details

# ------------------------------------
# DRIVER
# ------------------------------------
options = uc.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--start-maximized')
options.add_argument('--no-first-run')
options.add_argument('--no-default-browser-check')
options.add_argument('--user-data-dir=C:/chrome-profiles/fb-marketplace')

# ปล่อยให้ undetected_chromedriver เลือกเวอร์ชันอัตโนมัติ
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

# บังคับภาษา EN (selector คงที่ขึ้น) — ถ้าข้อมูลโพสต์เป็นไทยก็ยังอ่านได้
driver.get("https://www.facebook.com/?locale=en_US")
handle_login_popup(driver, wait)

# ------------------------------------
# READ URL LIST
# ------------------------------------
with open(INPUT_CSV_FILE, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader, None)  # หากไม่มี header จะไม่พัง
    urls_to_scrape = [row[0].strip() for row in reader if row and row[0].strip()]

# ------------------------------------
# SCRAPE
# ------------------------------------
all_details = []
for i, url in enumerate(urls_to_scrape):
    print(f"\nProcessing URL {i+1}/{len(urls_to_scrape)}: {url}")
    try:
        details = scrape_marketplace_details(driver, url)
        all_details.append(details)
    except Exception as e:
        print(f"Failed at {url}: {e}")
    time.sleep(2.5)

driver.quit()

# ------------------------------------
# WRITE OUTPUT
# ------------------------------------
if all_details:
    print(f"\nScraping complete. Writing {len(all_details)} records to {OUTPUT_CSV_FILE}...")
    headers = ['URL', 'Title', 'Price', 'Property_Details', 'Description']
    with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        writer.writeheader()
        writer.writerows(all_details)
    print("Process complete.")
else:
    print("No data scraped.")


Processing URL 1/500: https://www.facebook.com/marketplace/item/1003344581776669/

Processing URL 2/500: https://www.facebook.com/marketplace/item/1028021192836479/

Processing URL 3/500: https://www.facebook.com/marketplace/item/1034642708835304/

Processing URL 4/500: https://www.facebook.com/marketplace/item/1036231202045056/

Processing URL 5/500: https://www.facebook.com/marketplace/item/1048701880712043/

Processing URL 6/500: https://www.facebook.com/marketplace/item/1051538416875574/

Processing URL 7/500: https://www.facebook.com/marketplace/item/1054348250221902/

Processing URL 8/500: https://www.facebook.com/marketplace/item/1058315992852270/

Processing URL 9/500: https://www.facebook.com/marketplace/item/1063960362160988/

Processing URL 10/500: https://www.facebook.com/marketplace/item/1068372042130221/

Processing URL 11/500: https://www.facebook.com/marketplace/item/1072041375005784/

Processing URL 12/500: https://www.facebook.com/marketplace/item/1082095960560411/



In [ ]:
# # clean_fb_urls.py
# import os
# import re
# import csv
# import glob

# OUTPUT_PATH = r"Scraping\facebook_marketplace_urls.csv"

# def clean_url(u: str) -> str:
#     if not u:
#         return u
#     u = u.strip()

#     # ลบส่วนที่ซ้ำ "https://www.facebook.comhttps://"
#     u = re.sub(r'^https://(?:www\.|m\.)?facebook\.comhttps://', 'https://', u, flags=re.IGNORECASE)
#     u = re.sub(r'^http://(?:www\.|m\.)?facebook\.comhttps://', 'https://', u, flags=re.IGNORECASE)

#     # กันเคสโดเมนติดกันซ้ำ
#     u = re.sub(r'^(https?://(?:www\.|m\.)?facebook\.com)https://', r'\1/', u, flags=re.IGNORECASE)

#     return u

# def find_input_csv(explicit_path: str | None) -> str:
#     """ถ้าไม่ได้ระบุไฟล์ จะลองหา raw_urls.csv หรือไฟล์ .csv อื่นที่มีคอลัมน์ MarketplaceURL"""
#     if explicit_path and os.path.exists(explicit_path):
#         return explicit_path

#     cand = "raw_urls.csv"
#     if os.path.exists(cand):
#         return cand

#     # หา .csv ไฟล์ใกล้เคียง
#     for path in glob.glob("*.csv"):
#         try:
#             with open(path, "r", encoding="utf-8-sig", newline="") as f:
#                 reader = csv.DictReader(f)
#                 if reader.fieldnames and "MarketplaceURL" in reader.fieldnames:
#                     return path
#         except Exception:
#             pass

#     raise FileNotFoundError(
#         "ไม่พบไฟล์อินพุต กรุณาระบุพาธด้วย --input หรือวางไฟล์ raw_urls.csv ไว้ในโฟลเดอร์เดียวกัน"
#     )

# def run(input_path: str | None = None, output_path: str = OUTPUT_PATH) -> int:
#     # เตรียมโฟลเดอร์ปลายทาง
#     out_dir = os.path.dirname(output_path)
#     if out_dir:
#         os.makedirs(out_dir, exist_ok=True)

#     # หาไฟล์อินพุต
#     src = find_input_csv(input_path)

#     cleaned_rows = []
#     seen = set()

#     with open(src, "r", encoding="utf-8-sig", newline="") as f:
#         reader = csv.DictReader(f)
#         if "MarketplaceURL" not in reader.fieldnames:
#             raise ValueError("ไม่พบคอลัมน์ 'MarketplaceURL' ในไฟล์อินพุต")

#         for row in reader:
#             raw = row.get("MarketplaceURL", "")
#             url = clean_url(raw)
#             if url and url not in seen:
#                 seen.add(url)
#                 cleaned_rows.append({"MarketplaceURL": url})

#     with open(output_path, "w", encoding="utf-8", newline="") as f:
#         writer = csv.DictWriter(f, fieldnames=["MarketplaceURL"])
#         writer.writeheader()
#         writer.writerows(cleaned_rows)

#     print(f"บันทึกผลลัพธ์ {len(cleaned_rows)} แถว -> {output_path}")
#     return len(cleaned_rows)

# if __name__ == "__main__":
#     # พยายามอ่านอาร์กิวเมนต์แบบไม่บังคับ (กัน SystemExit ใน Notebook)
#     input_arg = None
#     try:
#         import argparse
#         parser = argparse.ArgumentParser(description="Clean duplicated 'https://www.facebook.comhttps://' in MarketplaceURL.")
#         parser.add_argument("--input", required=False, help="พาธไฟล์ CSV ที่มีคอลัมน์ 'MarketplaceURL'")
#         args, _ = parser.parse_known_args()
#         input_arg = args.input
#     except Exception:
#         input_arg = None

#     run(input_arg, OUTPUT_PATH)


บันทึกผลลัพธ์ 500 แถว -> Scraping\facebook_marketplace_urls.csv
